Initialise, set the path and import librarys

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
currPth = os.getcwd()
print(currPth)
import sys
sys.path.append("//home/ubuntu/fastai/") # go to parent dir

    (For some reason I have to tell FastAI where the parent dir is)

Import everything needed

In [ ]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.dataset import *
from fastai.dataset import *
from fastai.plots import *

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

from IPython.display import HTML

In [ ]:
PATH="//home/ubuntu/fastai/courses/data/EnergyDLModel/"

Set the Training & Testing data sets, import them from csv, parse the Date Column, have a look at it

In [ ]:
data_Train = pd.read_csv(f'{PATH}Train.csv', parse_dates=['Date'])
data_Train.head(20)

In [ ]:
data_Test = pd.read_csv(f'{PATH}Test.csv', parse_dates=['Date'])
data_Test.head(20)

Check lengths of data set

In [ ]:
len(data_Train)

In [ ]:
len(data_Test)

Add Extra Date information

In [ ]:
add_datepart(data_Train, "Date", drop=False)

In [ ]:
add_datepart(data_Test, "Date", drop=False)

Adjust Columns, discard unnecessary data

In [ ]:
data_Train.columns

In [ ]:
data_Test.columns

In [ ]:
columns = ['Date', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'Elapsed']

In [ ]:
data_Train.drop(columns,1,inplace=True)

In [ ]:
data_Test.drop(columns,1,inplace=True)

In [ ]:
data_Train.columns

In [ ]:
data_Test.columns

Backup to feather

In [ ]:
data_Train.reset_index(inplace=True)
data_Test.reset_index(inplace=True)

In [ ]:
data_Train.to_feather(f'{PATH}df')
data_Test.to_feather(f'{PATH}df_test')

Load the data from Feather

In [ ]:
data_Train = pd.read_feather(f'{PATH}df')

In [ ]:
data_Test= pd.read_feather(f'{PATH}df_test')

Convert DateTime to datetime for both datasets

In [ ]:
data_Train["DateTime"] = pd.to_datetime(data_Train.DateTime)

In [ ]:
data_Test["DateTime"] = pd.to_datetime(data_Test.DateTime)

Check data

In [ ]:
data_Train.columns

In [ ]:
data_Train.head().T.head(12)

Set Category and Continuous Variables

In [ ]:
cat_vars = ['Year', 'Month', 'Day', 'Dayofweek', 'Week', 'Dayofyear']
contin_vars = ['Demand','MaxTemp', 'MinTemp','Rainfall']
n = len(data_Train); n

In [ ]:
dep = 'Price'
data_Train = data_Train[cat_vars+contin_vars+[dep, 'DateTime']].copy()

In [ ]:
data_Test[dep] = 0
data_Test = data_Test[cat_vars+contin_vars+[dep, 'DateTime', 'Id']].copy()

In [ ]:
for v in cat_vars: data_Train[v] = data_Train[v].astype('category').cat.as_ordered()

In [ ]:
data_Test.columns

In [ ]:
data_Train.columns

In [ ]:
apply_cats(data_Test, data_Train)

Set Continuous Variables to Float32

In [ ]:
for v in contin_vars:
    data_Train[v] = data_Train[v].astype('float32')
    data_Test[v] = data_Test[v].astype('float32')

Set DateTime as index for both

In [ ]:
data_Train = data_Train.set_index("DateTime")

In [ ]:
data_Test = data_Test.set_index("DateTime")

Split off response variable (Price) and convert dataframe to entirely numeric data

In [ ]:
df, y, nas, mapper = proc_df(data_Train, 'Price', do_scale=True)
yl = np.log(y)

In [ ]:
df_test, _, nas, mapper = proc_df(data_Test, 'Price', do_scale=True, skip_flds=['Id'],
                                  mapper=mapper, na_dict=nas)

Set validation set to last 25% of Training set

In [ ]:
samp_size = len(df)

In [ ]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

Define Root-mean-squared percent error to use as metric

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

Create ModelData from DataFrame

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=64,
                                      test_df=df_test)

Use the cardinal for each Categorical variable for embeddings

In [ ]:
cat_sz = [(c, len(data_Train[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

Find a Learning Rate

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),0.04, 1, [1000,500], [0.001,0.01],y_range=y_range)
lr = 1e-3

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot(100)

Have a Look at the model

In [ ]:
m

Do some initial learning

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-4

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])